# Resolving

Resolvers are helpers to find commonly used resources that one may want to link resources to. Resolvers are configured in the `Resolvers` section of the configuration file.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](https://github.com/BlueBrain/nexus-forge/blob/master/examples/notebooks/nexus-demo/00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/v1) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
# Get a KnowledgeGraphForge session
forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

## Imports

In [ ]:
from kgforge.core.commons.strategies import ResolvingStrategy

## Discover scopes, resolvers and targets

With the `forge.resolvers()` method, the user can inspect the available scopes, resolvers, and targets.
A scope is linked to a data source (directory, store, or URL). Each scope will have at least one resolver, which is an implementation of the Forge Resolver. A resolver can have multiple targets, and it will delimitate the resolution within the resolver's scope.

In [ ]:
forge.resolvers()

## Ontology terms

To find resources using resolver, you can use regex expressions within the provided string.

### using BEST_MATCH strategy (default)

Use ontology terms to retrieve the "Primary somatosensory area, trunk, layer 1" using its notation "ssp-tr1":

In [ ]:
brain_region = forge.resolve("ssp-tr1", scope="ontology", target="terms")

In [ ]:
type(brain_region)

In [ ]:
print(brain_region)

If there is a unique scope, resolver, or target, it is possible to omit them in the parametrization.

In [ ]:
result = forge.resolve("ssp-tr1", scope="ontology")

In [ ]:
print(result)

In [ ]:
brain_region == result

### using ALL_MATCHES strategy

Use ontology terms to retreieve a "somatosensory" followed by "trunk" brain region:

In [ ]:
brain_regions = forge.resolve("somatosensory.*trunk", scope="ontology", strategy=ResolvingStrategy.ALL_MATCHES, limit=3)

In [ ]:
for x in brain_regions:
    print(x)

### using EXACT_MATCH strategy

For this strategy, the exact string has to be provided, and it is case sensitive.

In [ ]:
result = forge.resolve("Somatosensory areas, layer 1", scope="ontology", strategy=ResolvingStrategy.EXACT_MATCH)

In [ ]:
print(result)

## Agent Entities

### scope
Resolving an entity resource having "anne".

In [ ]:
agent = forge.resolve("anne", scope="agent")

In [ ]:
type(agent)

In [ ]:
print(agent)

### using ALL_MATCHES strategy

In [ ]:
agents = forge.resolve("anne", scope="agent", strategy=ResolvingStrategy.ALL_MATCHES)

In [ ]:
for x in agents:
    print(x)

### using EXACT_MATCH strategy

In [ ]:
agent = forge.resolve("anne", scope="agent", strategy=ResolvingStrategy.EXACT_MATCH)

In [ ]:
print(agent)

## filtering by type

In [ ]:
person = forge.resolve("anne", scope="agent", type="Person")

In [ ]:
print(person)

In [ ]:
organization = forge.resolve("anne", scope="agent", type="Organization")

In [ ]:
print(organization)